In [1]:
!nvidia-smi

Fri Dec 23 23:24:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 45%   30C    P8    N/A /  75W |    469MiB /  4096MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow as tf

gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

# Obraz - transfer learning - praca domowa
Ostatnia aktualizacja: 2022.12.10

Z dokumentacji [Keras](https://keras.io/guides/transfer_learning/): *Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.*

Głównym celem pracy domowej jest stworzenie klasyfikatora, który będzie odróżniał zdjęcia obiektów w wybranym przez nas zbiorze. Użyjemy do tego jednego z [gotowych modeli Keras](https://keras.io/api/applications/), wytrenowanego wcześniej na zbiorze Imagenet. 

Przydatne źródła:
- [transfer learning vgg16 + tf_flowers](https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4)
- [Keras - transfer learning](https://keras.io/guides/transfer_learning/)

### Wybrany zbiór danych:

- [znaki drogowe](https://www.kaggle.com/datasets/valentynsichkar/traffic-signs-1-million-images-for-classification). 


In [2]:
import h5py
filename = "dataset_ts_original.hdf5"

with h5py.File(filename, "r") as f:
    
    x_test = f['x_test'][()]
    x_train = f['x_train'][()]
    y_test = f['y_test'][()]
    y_train = f['y_train'][()]
    x_validation = f['x_validation'][()]
    y_validation = f['y_validation'][()]

### Zadanie 1: Wytrenuj model na swoich danych ###
**(Zadanie na ocenę 3)**

Wytrenuj wybrany model na swoich danych. Omów eksperyment i wyniki (100 słów). 

In [15]:
assert x_test.shape[1:] == x_train.shape[1:] == x_validation.shape[1:]
print(x_test.shape)
print(x_train.shape)
print(x_validation.shape)

(3111, 48, 48, 3)
(36288, 48, 48, 3)
(12440, 48, 48, 3)


In [6]:
# x_validation_resized = tf.image.resize(x_validation[:300], (150, 150)).numpy()
# x_test_resized = tf.image.resize(x_test[:50], (150, 150)).numpy()
# x_train_resized = tf.image.resize(x_train[:1500], (150, 150)).numpy()

In [7]:
x_train[0].shape

(48, 48, 3)

### Wczytaj wytrenowany model do klasyfikacji obrazu. ###
Może to być jeden z gotowych [modeli dostępnych w Keras](https://keras.io/api/applications/). Wczytujemy go z wytrenowanymi już wcześniej wagami na Imagenecie (weights='imagenet'). 

Model możemy wczytać bez ostatnich warstw (include_top=False) i dodać je potem ręcznie, dostosowane do liczby klas w naszym zbiorze. Imagenet ma 1000 klas, my prawdopodobnie będziemy mieć ich mniej. 

Pamiętaj, żeby wyłączyć lub ograniczyć trening części modelu z wytrenowanymi już wagami (trainable=False). 

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

base_model = VGG16(weights='imagenet', include_top=False, input_shape=x_train[0].shape)
base_model.trainable = False


In [10]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(150, activation='relu')
prediction_layer = layers.Dense(43, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    prediction_layer
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 150)               76950     
                                                                 
 dense_1 (Dense)             (None, 43)                6493      
                                                                 
Total params: 14,798,131
Trainable params: 83,443
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
y_train.shape

(36288,)

In [13]:
!pip install pandas

In [82]:
import pandas as pd
y_train = pd.get_dummies(y_train)
y_validation = pd.get_dummies(y_validation)

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
908/908 [==============================] - 41s 40ms/step - loss: 2.0640 - accuracy: 0.6008 - val_loss: 1.0568 - val_accuracy: 0.7152
Epoch 2/50
908/908 [==============================] - 35s 39ms/step - loss: 0.7532 - accuracy: 0.7740 - val_loss: 0.7749 - val_accuracy: 0.7852
Epoch 3/50
908/908 [==============================] - 35s 39ms/step - loss: 0.5510 - accuracy: 0.8295 - val_loss: 0.7730 - val_accuracy: 0.7834
Epoch 4/50
908/908 [==============================] - 35s 39ms/step - loss: 0.4541 - accuracy: 0.8551 - val_loss: 0.6933 - val_accuracy: 0.8143
Epoch 5/50
908/908 [==============================] - 35s 39ms/step - loss: 0.3894 - accuracy: 0.8763 - val_loss: 0.6481 - val_accuracy: 0.8330
Epoch 6/50
908/908 [==============================] - 35s 39ms/step - loss: 0.3391 - accuracy: 0.8892 - val_loss: 0.6649 - val_accuracy: 0.8325
Epoch 7/50
908/908 [==============================] - 35s 39ms/step - loss: 0.3023 - accuracy: 0.9006 - val_loss: 0.6673 - val_accuracy:

In [19]:
model.predict(x_train[:2])

1/1 [==============================] - 0s 23ms/step


array([[0.0000000e+00, 3.4126278e-36, 2.4456801e-34, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.3031604e-32,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 7.4905679e-31, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 1.0549303e-37, 0.0000000e+00,
        6.7797821e-36, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        4.5613204e-38, 0.0000000e+00, 7.4406331e-34, 0.0000000e+00,
        0.0000000e+00, 7.9408963e-36, 0.0000000e+00],
       [4.1512739e-12, 6.6080820e-06, 7.8983510e-07, 1.5811435e-06,
        1.3831764e-05, 3.1484937e-08, 3.7811067e-11, 2.5975558e-10,
        3.4432971e-06, 2.7480095e-08, 1.0203946e-10, 5.3578051e-04,
        9.3616372e-06, 3.6711779e-06, 2.4849458e-08, 2.5991131

*TODO: omówienie (100 słów)*

omówienie will be here...

### Zadanie 2: Dodatkowe sieci ###
**(Zadanie na ocenę 4, po wykonaniu  zadania 1)**

Przeprowadź to samo na dwóch dodatkowych sieciach i omów wyniki (100 słów). 

Czyli jeśli w zadaniu 1 użyliśmy np. VGG to teraz wybieramy sobie np. ResNet i MobileNet. 

In [30]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                labels = labels.type(torch.LongTensor)   # casting to long
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [21]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [22]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn
from torch import optim
import torch

num_epochs = 5
feature_extract=True
model_ft = None
input_size = 0
num_classes = 43
criterion = nn.CrossEntropyLoss()
base_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
optimizer_ft = optim.SGD(base_model.parameters(), lr=0.001, momentum=0.9)

set_parameter_requires_grad(base_model, feature_extract)
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Linear(num_ftrs, num_classes)
input_size = 224
device = torch.device("cuda:0")

In [23]:
base_model = base_model.to(device)

params_to_update = base_model.parameters()
print("Params to learn:")

params_to_update = []
for name,param in base_model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [24]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader


tensor_x = torch.Tensor(x_train) # transform to torch tensor
tensor_y = torch.Tensor(y_train)

data_transforms = {
    'train'
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader


SyntaxError: invalid syntax (<ipython-input-24-d51e8e81edfd>, line 10)

In [25]:
import time
import copy

In [26]:
from torch.utils.data import DataLoader, TensorDataset
my_dataset_train = TensorDataset(torch.Tensor(x_train).permute(0, 3, 1, 2), torch.Tensor(y_train))
my_dataset_val = TensorDataset(torch.Tensor(x_validation).permute(0, 3, 1, 2), torch.Tensor(y_validation))

loader= {'train': DataLoader(my_dataset_train, batch_size=64), 'val': DataLoader(my_dataset_val, batch_size=64)}

In [31]:
model_ft, hist = train_model(base_model, loader, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/4
----------
train Loss: 3.7989 Acc: 0.0284
val Loss: 3.7957 Acc: 0.0277

Epoch 1/4
----------
train Loss: 3.7989 Acc: 0.0284
val Loss: 3.7957 Acc: 0.0277

Epoch 2/4
----------
train Loss: 3.7989 Acc: 0.0284
val Loss: 3.7957 Acc: 0.0277

Epoch 3/4
----------
train Loss: 3.7989 Acc: 0.0284
val Loss: 3.7957 Acc: 0.0277

Epoch 4/4
----------
train Loss: 3.7989 Acc: 0.0284
val Loss: 3.7957 Acc: 0.0277

Training complete in 2m 35s
Best val Acc: 0.027653


*TODO: omówienie (100 słów)*

### Zadanie 3: Trening od zera i porównanie ###
**(Zadanie na ocenę 5, po wykonaniu zadania 1 i 2)**

Spróbuj skonstruować swój własny model i wytrenować go 'od zera' na tych samych danych. Porównaj i omów swój ekeperyment i wyniki (100 słów).

reference: https://keras.io/api/applications/#usage-examples-for-image-classification-models

*TODO: omówienie (100 słów)*

To wszystko, dziękuję. Wypełniony notatnik zapisz jako .ipynb i oddaj w Teams. 